# Morningstar Pro - Entraînement avancé sur Colab

## Système complet de trading algorithmique avec données sociales

Ce notebook permet :
- De télécharger les données de marché (OHLCV) depuis un exchange crypto
- D'ajouter des indicateurs techniques avancés
- D'intégrer des données sociales (GitHub et Reddit)
- D'entraîner un modèle de deep learning pour le trading

## 1. Installation des dépendances

In [ ]:
# Installation des dépendances système et Python
!pip install -q tensorflow==2.12.0 pandas==1.5.3 numpy==1.23.5 ccxt==4.1.91 ta pyarrow scikit-learn asyncpraw tweepy aiohttp PyGithub praw

## 2. Configuration

In [ ]:
# Clonage du dépôt et ajout du chemin Morningstar
!git clone https://github.com/Cabrel10/eva001.git
import sys
sys.path.insert(0, '/content/eva001')

In [ ]:
# Sélection interactive des paires et des dates
import datetime
default_pairs = 'BTC/USDT,ETH/USDT,BNB/USDT,SOL/USDT'
pairs = input(f"Entrez les paires séparées par une virgule (exemple: {default_pairs}): ") or default_pairs
pairs = [p.strip() for p in pairs.split(',')]
start_date = input("Date de début (YYYY-MM-DD, défaut 2023-01-01): ") or '2023-01-01'
end_date = input("Date de fin (YYYY-MM-DD, défaut aujourd'hui): ") or str(datetime.date.today())

In [ ]:
# Configuration des APIs sociales
from github import Github
import praw

# Config GitHub (remplacer par ton token)
github_token = input("Entrez votre token GitHub (ou laissez vide pour désactiver): ") or None
gh = Github(github_token) if github_token else None

# Config Reddit (remplacer par tes credentials)
reddit = praw.Reddit(
    client_id=input("Reddit client_id: ") or None,
    client_secret=input("Reddit client_secret: ") or None,
    user_agent="Morningstar Data Collector"
)

## 3. Pipeline de données

In [ ]:
# Fonctions pour données sociales
def get_github_stats(repo_name):
    if not gh:
        return None, None, None, None, None
    try:
        repo = gh.get_repo(repo_name)
        return (
            repo.get_commits().totalCount,
            repo.stargazers_count,
            repo.forks_count,
            repo.get_issues(state='open').totalCount,
            repo.get_issues(state='closed').totalCount
        )
    except:
        return None, None, None, None, None

def get_reddit_sentiment(subreddit, pair):
    if not reddit:
        return None
    try:
        submissions = reddit.subreddit(subreddit).search(f"{pair} flair:Discussion", limit=10)
        return sum(1 for s in submissions if s.score > 0) / 10  # Ratio de posts positifs
    except:
        return None

In [ ]:
# Téléchargement des données de marché
from Morningstar.utils.data_manager import ExchangeDataManager
import pandas as pd
import asyncio

# Correction pour l'event loop Colab/Jupyter
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

async def fetch_data(pairs, timeframe='1d', start_date=None, end_date=None):
    exchange = ExchangeDataManager(exchange_name="kucoin")
    await exchange.load_markets_async()
    all_data = []
    for pair in pairs:
        print(f"Téléchargement {pair}...")
        df = await exchange.load_data(pair, timeframe, start_date, end_date)
        if not df.empty:
            df['pair'] = pair
            all_data.append(df)
    await exchange.close()
    if all_data:
        return pd.concat(all_data)
    else:
        raise ValueError("Aucune donnée téléchargée.")

raw_data = asyncio.get_event_loop().run_until_complete(fetch_data(pairs, '1h', start_date, end_date))

In [ ]:
# Prétraitement et sauvegarde
from Morningstar.utils.custom_indicators import add_technical_indicators

def prepare_dataset(df):
    # Réinitialiser l'index pour éviter les doublons
    df = df.reset_index(drop=True)
    
    # Ajouter les indicateurs techniques
    df = add_technical_indicators(df)
    
    # Récupérer les données sociales pour chaque paire
    for pair in df['pair'].unique():
        # Mappage des paires aux repos GitHub et subreddits
        repo_map = {
            'BTC/USDT': 'bitcoin/bitcoin',
            'ETH/USDT': 'ethereum/go-ethereum',
            'BNB/USDT': 'binance-chain/docs',
            'SOL/USDT': 'solana-labs/solana'
        }
        
        subreddit_map = {
            'BTC/USDT': 'Bitcoin',
            'ETH/USDT': 'ethereum',
            'BNB/USDT': 'binance',
            'SOL/USDT': 'solana'
        }
        
        if pair in repo_map:
            commits, stars, forks, issues_opened, issues_closed = get_github_stats(repo_map[pair])
            mask = df['pair'] == pair
            df.loc[mask, 'commits'] = commits
            df.loc[mask, 'stars'] = stars
            df.loc[mask, 'forks'] = forks
            df.loc[mask, 'issues_opened'] = issues_opened
            df.loc[mask, 'issues_closed'] = issues_closed
            
        if pair in subreddit_map:
            sentiment = get_reddit_sentiment(subreddit_map[pair], pair.split('/')[0])
            df.loc[df['pair'] == pair, 'reddit_sentiment'] = sentiment
    
    # Colonnes finales
    columns = [
        'open', 'high', 'low', 'close', 'volume', 'rsi', 'macd', 'macd_signal', 'macd_hist',
        'bb_upper', 'bb_middle', 'bb_lower', 'volume_ma', 'volume_anomaly', 'pair',
        'commits', 'stars', 'forks', 'issues_opened', 'issues_closed', 'reddit_sentiment', 'datetime'
    ]
    
    # S'assurer que toutes les colonnes existent
    for col in columns:
        if col not in df.columns:
            df[col] = None
            
    return df[columns]

data = prepare_dataset(raw_data)
data.to_parquet('full_dataset.parquet')
print(f"Dataset final: {data.shape}")

## 4. Entraînement du modèle

In [ ]:
# Entraînement du modèle Morningstar
import tensorflow as tf
from Morningstar.workflows.training_workflow import TrainingWorkflow
class ColabConfig:
    def __init__(self):
        self.time_window = 50
        self.features = data.columns.tolist()
        self.epochs = 200
        self.batch_size = 1024
        self.dataset_path = 'full_dataset.parquet'
colab_config = ColabConfig()
workflow = TrainingWorkflow(colab_config)
tf_dataset = workflow._prepare_dataset(data)
dataset_size = tf.data.experimental.cardinality(tf_dataset).numpy()
val_size = int(dataset_size * 0.2)
train_dataset = tf_dataset.skip(val_size)
val_dataset = tf_dataset.take(val_size)
with tf.distribute.MirroredStrategy().scope():
    inputs = tf.keras.Input(shape=(50, len(data.columns)))
    x = tf.keras.layers.Conv1D(128, 5, activation='swish')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(128)(x)
    x = tf.keras.layers.Dense(64, activation='swish')(x)
    outputs = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=colab_config.epochs,
    batch_size=colab_config.batch_size,
    callbacks=callbacks
)

## 5. Sauvegarde finale

In [ ]:
# Sauvegarde finale et export sur Google Drive
model.save('morningstar_pro.h5')
from google.colab import drive
drive.mount('/content/drive')
!cp morningstar_pro.h5 '/content/drive/MyDrive/Colab Data/'